In [2]:
import pandas as pd
import os

# employment outlook 

In [102]:
# Load the cleaned data from the specified sheet
file_path = 'data/occupation-profiles-data.xlsx'
data = pd.read_excel(file_path, sheet_name='data_clean')


In [103]:
data.head()

,ANZSCO \ncode,Occupation,Employment_2011,Employment_2012,Employment_2013,Employment_2014,Employment_2015,Employment_2016,Employment_2017,Employment_2018,...,ACT (%),Age 15 - 19,Age 20 - 24,Age 25 - 34,Age 35 - 44,Age 45 - 54,Age 55 - 59,Age 60 - 64,Age 65 +,Median Age
0,1111,Chief Executives and Managing Directors,66200.0,69100.0,61600.0,46900.0,64000.0,51700.0,47400.0,63000.0,...,2.0,NaN,0.5,7.9,22.7,34.0,14.6,10.2,10.1,50
1,1112,General Managers,50300.0,49800.0,58100.0,53400.0,52800.0,46800.0,38900.0,46800.0,...,2.7,0.1,0.9,12.3,29.6,33.0,11.5,6.9,5.7,46
2,1113,Legislators,2100.0,1500.0,2500.0,1200.0,4700.0,1900.0,2700.0,3800.0,...,1.9,NaN,1.2,8.0,16.7,27.9,13.2,14.2,18.8,53
3,1211,Aquaculture Farmers,1600.0,1700.0,2000.0,1900.0,2300.0,4000.0,3900.0,1100.0,...,NaN,2.5,7.0,17.8,21.0,22.3,10.7,8.8,10.0,45
4,1212,Crop Farmers,45100.0,37200.0,47400.0,40900.0,42600.0,37400.0,43500.0,40100.0,...,0.1,0.8,3.0,11.8,17.8,23.9,12.7,11.3,18.7,52


In [104]:
data = data.rename(columns={'ANZSCO \ncode': 'ANZSCO code','Age 65 +  ':'Age 65 +'})

In [105]:
data.columns

Index(['ANZSCO code', 'Occupation', 'Employment_2011', 'Employment_2012',
       'Employment_2013', 'Employment_2014', 'Employment_2015',
       'Employment_2016', 'Employment_2017', 'Employment_2018',
       'Employment_2019', 'Employment_2020', 'Employment_2021',
       'Projected Employment level', 'Projected Employment Growth',
       'Projected Employment Growth (%)', 'Future Growth Rating', 'Male Share',
       'Female Share', 'Full-time Share', 'Part-time Share', 'industry',
       'NSW (%)', 'VIC (%)', 'QLD (%)', 'SA (%)', 'WA (%)', 'TAS (%)',
       'NT (%)', 'ACT (%)', 'Age 15 - 19', 'Age 20 - 24', 'Age 25 - 34',
       'Age 35 - 44', 'Age 45 - 54', 'Age 55 - 59', 'Age 60 - 64', 'Age 65 +',
       'Median Age'],
      dtype='object')

In [106]:
columns_to_keep = ['ANZSCO code', 'Occupation']
df_melted = pd.melt(data, 
                    id_vars=columns_to_keep, 
                    var_name="Year/Metric", 
                    value_name="Value")



In [107]:
# Display the first few rows to verify the transformation
data_melt.tail()


,ANZSCO code,Occupation,Year/Metric,Value,Metric,Year
13241,8994,Motor Vehicle Parts and Accessories Fitters,Median Age,34,Median Age,None
13242,8995,Printing Assistants and Table Workers,Median Age,46,Median Age,None
13243,8996,Recycling and Rubbish Collectors,Median Age,46,Median Age,None
13244,8997,Vending Machine Attendants,Median Age,37,Median Age,None
13245,8999,Other Miscellaneous Labourers,Median Age,43,Median Age,None


In [108]:
def split_year_metric(row):
    if 'Employment_' in row['Year/Metric']:
        metric, year = row['Year/Metric'].split('_')
        return pd.Series([year, metric])
    else:
        return pd.Series([None, row['Year/Metric']])

df_melted[['Year', 'Metric']] = df_melted.apply(split_year_metric, axis=1)


In [109]:
df_cleaned = df_melted[['ANZSCO code', 'Occupation', 'Year', 'Metric', 'Value']]


In [110]:
# value fill

# Fill missing values in 'Year' column where 'Metric' is 'Projected Employment level' with '2026'
df_cleaned.loc[(df_cleaned['Metric'] == 'Projected Employment level') & (df_cleaned['Year'].isnull()), 'Year'] = '2026'

In [111]:

# Display all rows where 'Value' is NaN
nan_rows = df_cleaned[df_cleaned['Value'].isna()]

# Display the result
nan_rows

,ANZSCO code,Occupation,Year,Metric,Value
25,1392,Senior Non-Commissioned Defence Force Officers,2011,Employment,NaN
223,4411,Defence Force Members - Other Ranks,2011,Employment,NaN
383,1392,Senior Non-Commissioned Defence Force Officers,2012,Employment,NaN
581,4411,Defence Force Members - Other Ranks,2012,Employment,NaN
741,1392,Senior Non-Commissioned Defence Force Officers,2013,Employment,NaN
...,...,...,...,...,...
10142,2541,Midwives,None,Age 15 - 19,NaN
10144,2543,Nurse Managers,None,Age 15 - 19,NaN
10153,2711,Barristers,None,Age 15 - 19,NaN
10155,2713,Solicitors,None,Age 15 - 19,NaN


In [112]:
# Fill NaN values in the 'Value' column with 0
df_cleaned['Value'] = df_cleaned['Value'].fillna(0)

/var/folders/j5/j2851kd10mq66kby8zkqyyf40000gn/T/ipykernel_34129/2091216732.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Value'] = df_cleaned['Value'].fillna(0)


In [113]:
df_cleaned.to_csv('data/employment_outlook_2.csv',index=False)